# [중요] 리소스 정리 (Clean-up Resource)
### 아래 과정을 꼼꼼하게 해주세요. 그렇지 않으면 요금이 발생할 수 있습니다.

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./2_Inference/src')

%store -r bucket

# 2. 세이지 메이커 앤드포인트 삭제 확인
- 아래와 같이 콘솔에 가셔서 엔드포인트가 없어야 합니다.
    - 만일 존재하면 삭제 해주세요.

- SageMaker Concole --> 왼쪽 메뉴의 Inference --> Endpoints 로 다음과 같이 엔드포인트가 있는지 확인 하세요.

![sm_endpoint_console.png](img/sm_endpoint_console.png)

In [2]:
from inference_utils import delete_endpoint
import boto3

In [3]:
client = boto3.Session().client('sagemaker')

endpoint_name = 'sm-ncf'

response = client.list_endpoints(
    SortBy='Name',
    SortOrder='Ascending',
    NameContains=endpoint_name,
)

for ep_list in response['Endpoints']:
    print("endpoint name: \n", ep_list["EndpointName"])
    ep = ep_list["EndpointName"]
    delete_endpoint(client, ep)

In [4]:
endpoint_name = 'lambda-deploy-endpoint'

response = client.list_endpoints(
    SortBy='Name',
    SortOrder='Ascending',
    NameContains=endpoint_name,
)

for ep_list in response['Endpoints']:
    print("endpoint name: \n", ep_list["EndpointName"])
    ep = ep_list["EndpointName"]
    delete_endpoint(client, ep)


In [5]:
endpoint_name = 'ncf-codepipeline-endpoint'

response = client.list_endpoints(
    SortBy='Name',
    SortOrder='Ascending',
    NameContains=endpoint_name,
)

for ep_list in response['Endpoints']:
    print("endpoint name: \n", ep_list["EndpointName"])
    ep = ep_list["EndpointName"]
    delete_endpoint(client, ep)


# 3. SageMaker 파이프라인 삭제

In [6]:
sagemaker_boto_client = boto3.client('sagemaker')

def clean_pipeline(pipeline_name, isDeletePipeline=False, verbose=False):
    '''
    파이프라인 삭제
    pipeline_name = 'sagemaker-pipeline-step-by-step-phase01'
    clean_pipeline(pipeline_name = pipeline_name, isDeletePipeline=False, verbose=False)   
    '''
    # project_prefix 의 prefix 를 가진 파이프라인을 모두 가져오기
    response = sagemaker_boto_client.list_pipelines(
        PipelineNamePrefix= pipeline_name,
        SortBy= 'Name',    
        SortOrder='Descending',
        #NextToken='string',
        MaxResults=100
    )

    if verbose:
        print(f"\n### Display pipelines with this prefix {pipeline_name} \n")        
        dp(response)

    
    # pipeline_name 보여주기
    if any(pipeline["PipelineDisplayName"] == pipeline_name for pipeline in response["PipelineSummaries"]):
        print(f"pipeline {pipeline_name} exists")
        response = sagemaker_boto_client.describe_pipeline(
            PipelineName= pipeline_name
        )    
    
        if verbose:
            print(f"\n### pipeline {pipeline_name} definiton is \n")
            dp(response)
            
        if isDeletePipeline:
            sagemaker_boto_client.delete_pipeline(PipelineName= pipeline_name)            
            print(f"pipeline {pipeline_name} is deleted")            

    else:
        print(f"pipeline {pipeline_name} doesn't exists")

## 3.1. 훈련 파이프라인 삭제 (1_sm_training_pipeline)

In [7]:
sm_training_pipeline = 'ncf-pipeline-nb-training'
clean_pipeline(pipeline_name = sm_training_pipeline, isDeletePipeline=True, verbose=False)   


pipeline ncf-pipeline-nb-training doesn't exists


## 3.2 추론 파이프라인 삭제 (2_sm_serving_pipeline)

In [8]:
sm_serving_pipeline = 'ncf-pipeline-nb-serving'
clean_pipeline(pipeline_name = sm_serving_pipeline, isDeletePipeline=True, verbose=False)   


pipeline ncf-pipeline-nb-serving doesn't exists


## 3.3. 코드파이프라인 연결 훈련 파이프라인 삭제 (4_sm-train-codepipeline)

In [9]:
sm_training_pipeline = 'ncf-training-code-pipeline-sm-pipeline'
clean_pipeline(pipeline_name = sm_training_pipeline, isDeletePipeline=True, verbose=False)   


pipeline ncf-training-code-pipeline-sm-pipeline doesn't exists


## 3.4. 코드파이프라인 연결 서빙 파이프라인 삭제 (5_sm-serving-codepipeline)

In [10]:
sm_serving_pipeline = 'ncf-serving-code-pipeline-sm-pipeline'
clean_pipeline(pipeline_name = sm_serving_pipeline, isDeletePipeline=True, verbose=False)   


pipeline ncf-serving-code-pipeline-sm-pipeline doesn't exists


# 4. 모델 훈련 EventBridge, Code Commit, Code Build, Code Pipeline 삭제 

## 4.1. 환경 설정

In [11]:
%store -r code_pipeline_train_config_json_path
%store -r sm_pipeline_train_config_json_path

from common_utils import load_json

code_pipeline_train_config_json_path = f'3_MLOps/4_sm_train_codepipeline/{code_pipeline_train_config_json_path}'
sm_pipeline_train_config_json_path = f'3_MLOps/4_sm_train_codepipeline/{sm_pipeline_train_config_json_path}'


code_pipeline_train_dict = load_json(code_pipeline_train_config_json_path)
sm_pipeline_train_dict = load_json(sm_pipeline_train_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_train_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_train_dict, indent=2))



Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/MLOps-CodePipelineRole",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/MLOps-CodeBuildRole",
  "project_prefix": "CodePipeline-Train-NCF",
  "region": "ap-northeast-2",
  "account_id": "057716757052",
  "train_code_repo_name": "ncf-training-repo",
  "code_build_project_name": "ncf-code-build-training-sm-pipeline",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "code_pipeline_name": "ncf-code-pipeline-training-sm-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "branch_name": "master",
  "update_time": "2022-12-30-02-02-55"
}
SageMaker Pipeline Series Params: 
{
  "project_prefix": "SageMaker-Train-NCF",
  "s3_input_data_uri": "s3://sagemaker-ap-northeast-2-057716757052/NCFModel/data2",
  "sm_pipeline_name": "ncf-training-code-pipeline-sm-pipeline",
  "training_instance_type": "ml.p3.2xlarge",
  "training_instance_count": 1,
  "ModelApprovalStatus": "Pend

In [12]:
repository_name = code_pipeline_train_dict["train_code_repo_name"]
code_build_project_name = code_pipeline_train_dict["code_build_project_name"]
code_pipeline_name = code_pipeline_train_dict["code_pipeline_name"]
model_package_group_name = code_pipeline_train_dict["model_package_group_name"]
bucket = code_pipeline_train_dict["bucket"]
sm_pipeline_name = sm_pipeline_train_dict["sm_pipeline_name"]

In [13]:
print("repository_name: ", repository_name)
print("code_build_project_name: ", code_build_project_name)
print("code_pipeline_name: ", code_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
print("bucket: ", bucket)
print("sm_pipeline_name: ", sm_pipeline_name)

repository_name:  ncf-training-repo
code_build_project_name:  ncf-code-build-training-sm-pipeline
code_pipeline_name:  ncf-code-pipeline-training-sm-pipeline
model_package_group_name:  NCF-Model-CodePipeline
bucket:  sagemaker-ap-northeast-2-057716757052
sm_pipeline_name:  ncf-training-code-pipeline-sm-pipeline


## 4.2 이벤트 브릿지 (타겟, 룰) 삭제


In [14]:
import boto3

client = boto3.Session().client('events')

eventbridge_rule_name1 = "codepipelinerule1"
codepipeline_name1 = "ncf-code-pipeline-training-sm-pipeline"

response = client.list_rules(NamePrefix=eventbridge_rule_name1)

# 이벤트 브릿지 룰이 있으면 룰 삭제
if 'Rules' in response and len(response['Rules']) > 0:
    response1 = client.remove_targets(
        Rule = eventbridge_rule_name1,
        Ids = [codepipeline_name1]
    )
    response1 = client.delete_rule(
        Name = eventbridge_rule_name1
    )
    print("EventBridge Rule ["+eventbridge_rule_name1+"] is deleted.")
else:
    print("There is no rule : "+eventbridge_rule_name1)

There is no rule : codepipelinerule1


## 4.3. 코드 리파지토리 삭제

In [15]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit delete-repository --repository-name $repository_name 


## 4.4. 코드 빌드 프로젝트 삭제

In [16]:
%%sh -s {code_build_project_name}
code_build_project_name=$1
aws codebuild delete-project --name $code_build_project_name

## 4.5. 코드 파이프라인 삭제

In [17]:
%%sh -s {code_pipeline_name}
pipeline_name=$1
aws codepipeline delete-pipeline --name $pipeline_name

# 5. 모델 서빙 EventBridge, Code Commit, Code Build, Code Pipeline 삭제 

## 5.1. 기존의 변수 불러오기

In [18]:
%store -r code_pipeline_serving_config_json_path
%store -r sm_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_config_json_path = f'3_MLOps/5_sm_serving_codepipeline/{code_pipeline_serving_config_json_path}'
sm_pipeline_serving_config_json_path = f'3_MLOps/5_sm_serving_codepipeline/{sm_pipeline_serving_config_json_path}'

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))

Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/MLOps-CodePipelineRole",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/MLOps-CodeBuildRole",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "ap-northeast-2",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving-repo",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master",
  "update_time": "2022-12-30-02-03-16"
}
SageMaker Pipeline Series Params: 
{
  "sm_pipeline_name": "ncf-serving-code-pipeline-sm-pipeline",
  "endpoint_instance_type": "ml.g4dn.xlarge",
  "ModelApprovalStatus": "Approved",
  "bucket": "sagemaker-ap-northeast-2-057716757052",
  "update_time": "2022-12-30-02-03-16"
}


In [19]:
import sagemaker
import boto3

repository_name = code_pipeline_serving_dict["serving_code_repo_name"]
code_build_project_name = code_pipeline_serving_dict["code_build_project_name"]
code_pipeline_name = code_pipeline_serving_dict["code_pipeline_name"]
endpoint_name = code_pipeline_serving_dict["endpoint_name"]
bucket = code_pipeline_serving_dict["bucket"]
model_package_group_name = code_pipeline_serving_dict["model_package_group_name"]

sm_pipeline_name = sm_pipeline_serving_dict["sm_pipeline_name"]

print("repository_name: ", repository_name)
print("bucket: ", bucket)
print("code_build_project_name: ", code_build_project_name)
print("sm_pipeline_name: ", sm_pipeline_name)
print("model_package_group_name: ", model_package_group_name)
print("endpoint_name: ", endpoint_name)

repository_name:  ncf-serving-repo
bucket:  sagemaker-ap-northeast-2-057716757052
code_build_project_name:  ncf-serving-sm-pipeline
sm_pipeline_name:  ncf-serving-code-pipeline-sm-pipeline
model_package_group_name:  NCF-Model-CodePipeline
endpoint_name:  ncf-codepipeline-endpoint


## 5.2 이벤트 브릿지 (타겟, 룰) 삭제

In [20]:
import boto3

client = boto3.Session().client('events')

eventbridge_rule_name2 = "codepipelinerule2"
codepipeline_name2 = "ncf-serving-code-pipeline"

response = client.list_rules(NamePrefix=eventbridge_rule_name2)

# 이벤트 브릿지 룰이 있으면 룰 삭제
if 'Rules' in response and len(response['Rules']) > 0:
    response2 = client.remove_targets(
        Rule = eventbridge_rule_name2,
        Ids = [codepipeline_name2]
    )
    response2 = client.delete_rule(
        Name = eventbridge_rule_name2
    )
    print("EventBridge Rule ["+eventbridge_rule_name2+"] is deleted.")
else:
    print("There is no rule : "+eventbridge_rule_name2)

There is no rule : codepipelinerule2


## 5.3. 코드 리파지토리 삭제

In [21]:
%%sh -s {repository_name}
repository_name=$1
aws codecommit delete-repository --repository-name $repository_name 


## 5.4. 코드 빌드 프로젝트 삭제

In [22]:
%%sh -s {code_build_project_name}
code_build_project_name=$1
aws codebuild delete-project --name $code_build_project_name

## 5.5. 코드 파이프라인 삭제

In [23]:
%%sh -s {code_pipeline_name}
pipeline_name=$1
aws codepipeline delete-pipeline --name $pipeline_name

# 6. 모델 패키지 삭제

- 해당 패키지 그룹을 삭제 함.
- 추가적으로 다른 채키지 그룹은 리스트로 보여 줌.

In [24]:
sm_client = boto3.Session().client('sagemaker')

def delete_model_package_group(sm_client, ModelPackageGroupName, isDelete=False, verbose=True):

    print("Current Model Package Group: \n")    
    for mpg in sm_client.list_model_package_groups()['ModelPackageGroupSummaryList']:
        print(mpg['ModelPackageGroupName'])

        if (mpg['ModelPackageGroupName'] == ModelPackageGroupName): # 해당 패키지만 그룹만 삭제
            for mp in sm_client.list_model_packages(ModelPackageGroupName=mpg['ModelPackageGroupName'])['ModelPackageSummaryList']:
                print(mp['ModelPackageArn'])
                try:
                    if isDelete:
                        sm_client.delete_model_package(ModelPackageName=mp['ModelPackageArn'])
                        print(f"###### ModelPackageGroupName: {ModelPackageGroupName} is deleted ######")
                except BaseException as error:
                    error_msg = f"Model Package error occurred: {error}"
                    print(error_msg)  
                    pass

            try:
                if isDelete:
                    sm_client.delete_model_package_group(ModelPackageGroupName=mpg['ModelPackageGroupName'])
            except:
                print("model_pakcage_group error")
                pass
        


In [25]:
model_package_group_name = 'NCFModel'
delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=True, verbose=True) 

Current Model Package Group: 

sagemaker-webinar-pipeline-base
model-building-training-p-6po9o9c7aofo
sagemaker-webinar-pipeline-advanced
sagemaker-pipeline-step-by-step-phase02
tuning-job-model-packages
sagemaker-pipeline-phase2-step-by-step
sagemaker-pipeline-step-by-step
fraud-detect-demo
only-train-template-p-a0vph0bcavzl
demo-project-gsmoon-p-9mvtktb4j47f


In [26]:
model_package_group_name = 'NCF-Model-CodePipeline'
delete_model_package_group(sm_client, ModelPackageGroupName=model_package_group_name, isDelete=True, verbose=True) 

Current Model Package Group: 

sagemaker-webinar-pipeline-base
model-building-training-p-6po9o9c7aofo
sagemaker-webinar-pipeline-advanced
sagemaker-pipeline-step-by-step-phase02
tuning-job-model-packages
sagemaker-pipeline-phase2-step-by-step
sagemaker-pipeline-step-by-step
fraud-detect-demo
only-train-template-p-a0vph0bcavzl
demo-project-gsmoon-p-9mvtktb4j47f


# 7. 세이지 메이커 실험 지우기

In [27]:
import time

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

In [28]:
def delete_experiment(experiment_name):
    try:
        experiment_to_cleanup = Experiment.load(
            # Use experiment name not display name
            experiment_name=experiment_name)

        cleanup_sme_sdk(experiment_to_cleanup)
    except Exception:
        print(f"There is no experiment, {experiment_name}")    

def cleanup_sme_sdk(experiment):
    try:
        for trial_summary in experiment.list_trials():
            trial = Trial.load(trial_name=trial_summary.trial_name)
            for trial_component_summary in trial.list_trial_components():
                tc = TrialComponent.load(
                    trial_component_name=trial_component_summary.trial_component_name)
                trial.remove_trial_component(tc)
                try:
                    # comment out to keep trial components
                    tc.delete()
                except:
                    # tc is associated with another trial
                    continue
                # to prevent throttling
                time.sleep(.5)
            trial.delete()
            experiment_name = experiment.experiment_name
        experiment.delete()
        print(f"\nExperiment {experiment_name} deleted")
    except Exception:
        print(f"There is no experiment, {experiment}")    

        

In [29]:
# 첫번째 생성된 실험
experiment_name='ncf-pipeline-nb-training'  
delete_experiment(experiment_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


There is no experiment, ncf-pipeline-nb-training


In [30]:
# 두번째 생성된 실험
experiment_name='ncf-training-code-pipeline-sm-pipeline' 
delete_experiment(experiment_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


There is no experiment, ncf-training-code-pipeline-sm-pipeline


In [31]:
# 세번째 생성된 실험
experiment_name='ncf-serving-code-pipeline-sm-pipeline' 
delete_experiment(experiment_name)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


There is no experiment, ncf-serving-code-pipeline-sm-pipeline


# 8. 람다 함수 삭제
- 생성된 3개의 함수 삭제 합니다.
    - function_name = 'sagemaker-lambda-step-approve-model-deployment'
    - function_name = 'sagemaker-lambda-step-endpoint-deployment'    
    - function_name = 'sagemaker-lambda-step-repackage-model-artifact'        

In [32]:
def delete_lambda_func(function_name):
    try:
        response = client.get_function(
            FunctionName=function_name,
        #    Qualifier='string'
        )
        print(f"function, {function_name}, exists")            
        response = client.delete_function(
            FunctionName= function_name,
        )        
        print(f"function, {function_name}, is deleted")            
    except Exception:
        print(f"There is no function, {function_name}")    


In [33]:
# 1st 함수 삭제
function_name = 'sagemaker-lambda-step-approve-model-deployment'
delete_lambda_func(function_name)

There is no function, sagemaker-lambda-step-approve-model-deployment


In [34]:
# 2nd 함수 삭제
function_name = 'sagemaker-lambda-step-endpoint-deployment'    
delete_lambda_func(function_name)

There is no function, sagemaker-lambda-step-endpoint-deployment


In [35]:
# 3rd 함수 삭제
function_name = 'sagemaker-lambda-step-repackage-model-artifact'        
delete_lambda_func(function_name)

There is no function, sagemaker-lambda-step-repackage-model-artifact


# 9. S3 버킷 삭제
아래는 사용한 버켓 (예: sagemaker-us-east-1-XXXXXX) 에서 생성된 폴더 입니다. 

![created_bucket.png](img/created_bucket.png)

## 아래는 NCFModel 폴더 한개 만을 보여 주고 삭제 하는 예시
- 다른 폴더도 삭제를 원하시면 해주세요.

In [36]:
!aws s3 ls s3://{bucket}/NCFModel --recursive

In [37]:
!aws s3 rm s3://{bucket}/NCFModel --recursive

In [38]:
!aws s3 ls s3://{bucket}/NCFModel --recursive

# 10. 세이지 메이커 노트북 인스턴스 삭제
## 처음 실습시 생성한 클라우드 포메이션으로 생성한 스택을 삭제를 해주세요. 다음 가이드를 진행 해주세요.
- 아래 링크 누르시고, "2. 역할, 권한 및 SageMaker Notebook 을 삭제" 를 진행 해주세요.
    - [여기](Setup_Environment/Cleanup-README.md) 링크 누르세요.


## [참고] 아래는 수동 삭제의 예시 입니다.
- 노트북 인스턴스를 선택하고, Stop (중지) 후에 Delete(삭제) 해주세요.

![notebook_instance.png](img/notebook_instance.png)

## 아래와 같이 위의 노트북 인스턴스가 없어야 합니다.

![empty_notebook.png](img/empty_notebook.png)